In [1]:
#import relevant libraries: pip install re, pip install natsort, pip install plotly==5.10.0
import sys
import os
import glob

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import datetime as dt8
import math
import matplotlib.pyplot as plt
import decimal
import re
from natsort import index_natsorted
import dabest
import dabest_jck
import itertools
import NLCLIMB 
import NLGRAPHS
import NLMATH

import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objects import Layout
from scipy.optimize import curve_fit

#NOTE: SUPPRESSES WARNINGS!

import warnings

warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [19]:
#initial file processing

computer1 = "C:\\Users\\User"
computer2 = "C:\\Users\\lnico"
filedir = "\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\Data Compilation\\Falling_New\\"
openPath = "C:\\Users\\User\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\eOPN3\\Tempfolder\\Compiled\\"
#openPath = computer1 + filedir

# openPath = "C:\\Users\\User\\ACC Lab Dropbox\\ACC Lab\\Zhiyi\\NL's amended data\\Compilation\\"
files = os.listdir(openPath)

#identifying genotypes
responder = "eOPN3;Gao"
driver = "OK371"
wt = "w1118"
transgenic = driver + " x " + responder
filename = openPath + transgenic + ".csv"
filenamewt = openPath + wt+"_"+ transgenic + ".csv"

print(filename)
print(filenamewt)

dfe=pd.read_csv(filename)
dfw= pd.read_csv(filenamewt)

exptdf = dfe.drop(dfe.columns[[0]],axis = 1)
wtdf = dfw.drop(dfw.columns[[0]],axis = 1)

dfexpt = NLCLIMB.generation(exptdf, driver)
dfwt = NLCLIMB.generation(wtdf, wt)

C:\Users\User\ACC Lab Dropbox\ACC Lab\Nicole Lee\eOPN3\Tempfolder\Compiled\OK371 x eOPN3;Gao.csv
C:\Users\User\ACC Lab Dropbox\ACC Lab\Nicole Lee\eOPN3\Tempfolder\Compiled\w1118_OK371 x eOPN3;Gao.csv


c:\Users\User\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

c:\Users\User\Documents\GitHub\deltaG_screen\NLCLIMB.py:376: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

c:\Users\User\Documents\GitHub\deltaG_screen\NLCLIMB.py:376: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

c:\Users\User\Documents\GitHub\deltaG_screen\NLCLIMB.py:376: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

c:\Users\User\Documents\GitHub\deltaG_screen\NLCLIMB.py:376: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.ilo

# functions

In [3]:
def countval(data, value):  #value = pause events
    count = 0
    timelst =[]
    for key, group in itertools.groupby(data, lambda x: x == value ):
        groupAsList = list(group)
        if( key == True ):
            count += 1
            timed = 0.2*len(groupAsList)
            timelst.append(timed)

        
    return (count, timelst)

def behavior (dfp):
   
    pc = dfp.filter(regex="Pausecount_.*")
    countpause = []
    countbout = []
    pcpause = pd.DataFrame()
    pcbout = pd.DataFrame()

    for n in pc:
        pc9 = pd.DataFrame()
        counter1, pausetime = countval(pc[n], 1) #pause = 1
        counter0, bouttime = countval(pc[n], 0) #bout = 0
        
        countpause.append(counter1)
        countbout.append(counter0)
        
        pcpause = pd.concat([pcpause, pd.Series(pausetime, dtype='float64')], ignore_index = True, axis = 1)
        pcbout = pd.concat([pcbout, pd.Series(bouttime, dtype='float64')], ignore_index = True, axis = 1)

    return countpause, countbout, pcpause, pcbout

def boutanalysis(df_dark, phase) : 
    
    countpause, countbout, pausedark, boutdark = behavior(df_dark)
    
    #avg paus time per fly (Mean Activity time spent per fly)
    meanpdark = pausedark.mean(axis = 0)
    meanbdark = boutdark.mean(axis = 0)
    meandarkevent = pd.DataFrame({"Pauses_" + phase: meanpdark, "Bouts_" + phase: meanbdark})
    
    #time per activity (raw_marker_size=0.5 ,swarm_label= "Time spent per activity")
    pausedarkdf = pausedark.melt().drop(['variable'], axis =1).dropna(axis = "index")
    boutdarkdf = boutdark.melt().drop(['variable'], axis =1).dropna(axis = "index")
    timedarkevent = pd.DataFrame({"Pauses_" + phase: pausedarkdf['value'], "Bouts_" + phase: boutdarkdf['value']})
    
    #occurences
    countevent = pd.DataFrame({"Pauses_" + phase: countpause, "Bouts_" + phase: countbout})
    
    return countevent, meandarkevent, timedarkevent

def pausecomp(dft, genre): #genre is either w1118, or driver line
    
    df_dark = dft[(dft['ExperimentState'] == 'Assimilation time - Dark') | (dft['ExperimentState']== 'Dark')] 
    df_light = dft[(dft['ExperimentState'] == 'Assimilation time - Full') | (dft['ExperimentState']== 'Full')] 
    #df_rec = dft[(dft['ExperimentState']== 'Recovery')]
    
    countdark, meandarkevent, timedarkevent  = boutanalysis(df_dark, "Dark")
    countlight, meanlightevent, timelightevent  = boutanalysis(df_light, "Full")
    #countrec, meanrecevent, timerecevent  = boutanalysis(df_rec, "Recovery")
    
    totalmeanevent = pd.concat([meandarkevent, meanlightevent], axis =1)
    totalmeanevent = totalmeanevent.add_prefix(genre + "_")
    
    totaltimeevent = pd.concat([timedarkevent, timelightevent], axis =1)
    totaltimeevent = totaltimeevent.add_prefix(genre + "_")
    
    totalnumberevent = pd.concat([countdark, countlight], axis =1)
    totalnumberevent = totalnumberevent.add_prefix(genre + "_")
    
    return totalmeanevent,totalnumberevent, totaltimeevent

def disttravel (dft, light, genre):    


    totaltravel = pd.DataFrame()

    totaldisttravelled = pd.DataFrame()
    distdf = (dft.filter(regex="Velocity.*"))*0.2
    totaldisttravelled = distdf.sum(axis= 0)

    lstdist = totaldisttravelled.index.tolist()
    lstdist = [s.replace("Velocity", "Dist") for s in lstdist]

    totaldisttravelled.index = lstdist

    
    totaltravel['value'] = totaldisttravelled
    totaltravel["ExperimentState"] = light
    totaltravel["Type"] = genre
    

    return totaltravel.reset_index()


In [4]:
def counterfall(df_rr1, nnumber, nnumberwt, phase): #phase is either "wt" or "expt"
    count_wtfull = df_rr1[(df_rr1["Type"] == phase)]['value'].value_counts()
    count_wtfull = count_wtfull.rename_axis('value').reset_index(name='counts')
    if phase == "WT":
        dvis = nnumberwt
    else:
        dvis = nnumber
    count_wtfull['counts'] = (count_wtfull['counts']/dvis)*100
    
    listofval = [x for x in [float(x) for x in range(0,23)] if x not in set(count_wtfull['value'])]
    count_wtfull = pd.concat([count_wtfull, pd.DataFrame({"value" :listofval})], ignore_index = True)
    
    count_wtfull = count_wtfull.sort_values(by=['value']).reset_index(drop = True)
    count_wtfull['counts'] = count_wtfull['counts'].replace(np.nan, 0)
    count_wtfull['name'] = phase
    
    return count_wtfull

def combinedgraph(df5, newtime):
    df5n = df5.copy()
    df5n['Seconds'] = newtime
    
    return df5n

def counterfall2(df, nnumber, nnumberwt, phase): #phase is either "wt" or "expt"
    
    totaldf = pd.DataFrame()
    typeo = ['Dark', 'Full', 'Recovery']
    for i in typeo:
        df_rr1 = df[(df["ExperimentState"] == i)]
        count_wtfull = df_rr1[(df_rr1["Type"] == phase)]['value'].value_counts()
        count_wtfull = count_wtfull.rename_axis('value').reset_index(name='counts')
        if phase == "WT":
            dvis = nnumberwt
        else:
            dvis = nnumber
        count_wtfull['counts'] = (count_wtfull['counts']/dvis)*100
        
        listofval = [x for x in [float(x) for x in range(0,23)] if x not in set(count_wtfull['value'])]
        count_wtfull = pd.concat([count_wtfull, pd.DataFrame({"value" :listofval})], ignore_index = True)
        
        count_wtfull = count_wtfull.sort_values(by=['value']).reset_index(drop = True)
        count_wtfull['counts'] = count_wtfull['counts'].replace(np.nan, 0)
        count_wtfull['name'] = phase
        count_wtfull['ExperimentState'] = i
        totaldf = pd.concat([totaldf, count_wtfull], axis = 0)
    return totaldf.reset_index(drop=True)

In [5]:
def removingextraline(df1):
    df = df1.reset_index(drop=True)
    df['Seconds'] = round(df['Seconds'],1)
    line = df[(df['Seconds'] > 22.0)].index
    df = df.drop(line)
    
    return df

In [16]:
def boutspeed(dfexpt):
    dfr = dfexpt.iloc[:,2:]
    velp = pd.DataFrame()
    velper = pd.DataFrame()
    for v2 in range(3,len(dfr.columns),5): #change this number if you add more parameters
        velp = pd.concat([velp, dfr.iloc[:,v2], dfr.iloc[:,v2+1]], axis = 1)

    velplst = []
    gentype = []

    for n in velp.columns[::2]:
        velplst.append(n.split("_")[1])
        gentype.append(n.split(" ")[0])

    newspeed = pd.DataFrame()

    for n,k in zip(velplst, gentype):
        newspeed[k + " BSpeed_" + n] = [np.nan]*len(velp)
        newspeed.loc[(velp[k + " Pausecount_" + n] ==0), [k + " BSpeed_" + n]] = velp[k + " Velocity_" + n]

    newspeed = pd.concat([dfexpt.iloc[:,0:2], newspeed], axis = 1)
    
    return newspeed

In [24]:
def boutheight(dfexpt):
    dfr = dfexpt.iloc[:,2:]
    velp = pd.DataFrame()
    velper = pd.DataFrame()
    for v2 in range(1,len(dfr.columns),5): #change this number if you add more parameters
        velp = pd.concat([velp, dfr.iloc[:,v2], dfr.iloc[:,v2+3]], axis = 1)

    velplst = []
    gentype = []

    for n in velp.columns[::2]:
        velplst.append(n.split("_")[1])
        gentype.append(n.split(" ")[0])

    newspeed = pd.DataFrame()

    for n,k in zip(velplst, gentype):
        newspeed[k + " Height_" + n] = [np.nan]*len(velp)
        newspeed.loc[(velp[k + " Pausecount_" + n] ==0), [k + " Height_" + n]] = velp[k + " Y_" + n]

    newspeed = pd.concat([dfexpt.iloc[:,0:2], newspeed], axis = 1)
    
    return newspeed

# **TOTAL calculations for graphs**



In [20]:
#Y position
df_de = NLMATH.calcgraph(dfexpt,"Y.*")
df_meande=NLMATH.meangraph(df_de)
df_dw = NLMATH.calcgraph(dfwt,"Y.*")
df_meandw=NLMATH.meangraph(df_dw)

#speed
df_se = NLMATH.calcgraph(dfexpt, "Velocity.*")
df_meanse = NLMATH.meangraph(df_se)
df_sw = NLMATH.calcgraph(dfwt, "Velocity.*")
df_meansw = NLMATH.meangraph(df_sw)

#Falling
df_fed = NLMATH.fallcalc(dfexpt, "Dark")
df_fel = NLMATH.fallcalc(dfexpt, "Full")
df_fer = NLMATH.fallcalc(dfexpt, "Recovery")

df_fwd = NLMATH.fallcalc(dfwt, "Dark")
df_fwl = NLMATH.fallcalc(dfwt, "Full")
df_fwr = NLMATH.fallcalc(dfwt, "Recovery")

#violinplots
# df_vfwd = NLMATH.violinfall(dfwt, "Dark")
# df_vfed = NLMATH.violinfall(dfexpt, "Dark")

# df_vfwf = NLMATH.violinfall(dfwt, "Full")
# df_vfef = NLMATH.violinfall(dfexpt, "Full")

#Falling
df_re = NLMATH.rastergraph(dfexpt)
df_rw = NLMATH.rastergraph(dfwt)
df_re['Type'] = "EXPT"
df_rw['Type'] = "WT"

df_rrdark = pd.concat([df_re[(df_re['ExperimentState']== 'Dark')], df_rw[(df_rw['ExperimentState']== 'Dark')]]).reset_index(drop=True)
#df_rr['list']= range(1,len(df_rr)+1)
df_rrfull = pd.concat([df_re[(df_re['ExperimentState']== 'Full')], df_rw[(df_rw['ExperimentState']== 'Full')]]).reset_index(drop=True)
#df_rr1['list']= range(1,len(df_rr1)+1)
df_rrrec = pd.concat([df_re[(df_re['ExperimentState']== 'Recovery')], df_rw[(df_rw['ExperimentState']== 'Recovery')]]).reset_index(drop=True)

c:\Users\User\Documents\GitHub\deltaG_screen\NLMATH.py:114: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[12.2 19.  19.8]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

c:\Users\User\Documents\GitHub\deltaG_screen\NLMATH.py:114: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[20.2]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

c:\Users\User\Documents\GitHub\deltaG_screen\NLMATH.py:114: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 6.8  8.   9.  13.4]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

c:\Users\User\Documents\GitHub\deltaG_screen\NLMATH.py:114: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a fu

# Compilation 2

In [7]:
df3 = removingextraline(df_meande)
df4 = removingextraline(df_meandw)
df5 = removingextraline(df_meanse)
df6 = removingextraline(df_meansw)
driver = transgenic

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objects import Layout

fig = make_subplots(rows=6, cols=2, row_heights=[0.02, 0.02, 0.02, 0.3,0.3, 0.3],shared_yaxes=True, shared_xaxes=True,vertical_spacing=0.05, horizontal_spacing = 0.05) #, subplot_titles=("Dark phase", "Light phase", "", " ")

if responder == "ACR":
    barcolr = "#00ff00"
if responder == "Chrimson2":
    barcolr = "#DC143C"

barcolr = "#00ff00"
#basic cartoon
fig.add_trace(go.Scatter(x=[], y=[]),row=1, col=1)
fig.add_shape(type="rect",x0=0, y0=0, x1=0.2, y1=2,line=dict(color="black",width=1,),fillcolor="gold",row=1,col=1)
fig.add_shape(type="rect",x0=0.2, y0=0, x1=3, y1=1,line=dict(color="black",width=2,),fillcolor="black",row=2,col=1)
fig.add_shape(type="rect",x0=3, y0=0, x1=23, y1=1,line=dict(color="black",width=2,),fillcolor="#7f7f7f",row=3,col=1)

fig.add_trace(go.Scatter(x=[], y=[]),row=1, col=2)
fig.add_shape(type="rect",x0=0, y0=0, x1=0.2, y1=2,line=dict(color="black",width=1,),fillcolor="gold",row=1,col=2)
fig.add_shape(type="rect",x0=0.2, y0=0, x1=3, y1=1,line=dict(color="black",width=2,),fillcolor="black",row=2,col=2)
fig.add_shape(type="rect",x0=3, y0=0, x1=23, y1=1,line=dict(color="black",width=2,),fillcolor=barcolr,row=3,col=2)
fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,1],row=1, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,2.1],row=1, col=1)
fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,1],row=1, col=2)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,2.4],row=1, col=2)

fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,3.05],row=2, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=2, col=1)
fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,3.05],row=2, col=2)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=2, col=2)

fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,23.05],row=3, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=3, col=1)
fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,23.05],row=3, col=2)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=3, col=2)


lightcolor = "#0055ff"
SDcolor = " #cce6ff"
SDinner = "rgba(179, 217, 255,0.3)"

nnumber = int(0.25*(len(dfexpt.columns)-2))
nnumberwt = int(0.25*(len(dfwt.columns)-2))

time = df3.loc[df3.ExperimentState == "Dark", 'Seconds']
time2 = df4.loc[df4.ExperimentState == "Dark", 'Seconds']
#expt_dark
row1 = 4
row2 = 5
row3 = 6
fig.append_trace(go.Scattergl(x =time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='Mean of Y position(mm)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),                                           
        showlegend=False   
    ), row=row1, col=1)

#expt_light
fig.append_trace(go.Scattergl(x =time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean of Y position(mm)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row1, col=2)

#exptdark_CI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean']+df3.loc[df3.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color= SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=1)
fig.append_trace(go.Scatter(
        name=transgenic,
        x=time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean']-df3.loc[df3.ExperimentState == "Dark", 'CI'],
        marker=dict(color= SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor= SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row1, col=1)

#exptlight_CI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean']+df3.loc[df3.ExperimentState == "Full", 'CI'],
        mode='lines',
        marker=dict(color=SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=2)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean']-df3.loc[df3.ExperimentState == "Full", 'CI'],
        marker=dict(color=SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row1, col=2)

#wt dark mean
fig.append_trace(go.Scattergl(x =time2,
        y=df4.loc[df4.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='WT',
        marker = dict(color = "#000"),
        line=dict(width=2),
        showlegend=False                              
    ), row=row1, col=1)

#wt light mean
fig.append_trace(go.Scattergl(x =time2,
        y=df4.loc[df4.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean of Y position(mm)_WT',
        marker = dict(color = "#000"),
        line=dict(width=2),
        showlegend=False        
    ), row=row1, col=2)

#wtdarkCI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time2,
        y=df4.loc[df4.ExperimentState == "Dark", 'mean']+df4.loc[df4.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=1)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time2,
        y=df4.loc[df4.ExperimentState == "Dark", 'mean']-df4.loc[df4.ExperimentState == "Dark", 'CI'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), row=row1, col=1)

#wtlightCI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time2,
        y=df4.loc[df4.ExperimentState == "Full", 'mean']+df4.loc[df4.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=2)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time2,
        y=df4.loc[df4.ExperimentState == "Full", 'mean']-df4.loc[df4.ExperimentState == "Dark", 'CI'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), row=row1, col=2)




#speed


time = df5.loc[df5.ExperimentState == "Dark", 'Seconds']
time2 = df6.loc[df6.ExperimentState == "Dark", 'Seconds']
#exptdark
fig.append_trace(go.Scattergl(x =time,
        y=df5.loc[df5.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row2, col=1)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Dark", 'mean']+df5.loc[df5.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color= SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=1)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=df5['Seconds'],
        y=df5.loc[df5.ExperimentState == "Dark", 'mean']-df5.loc[df5.ExperimentState == "Dark", 'CI'],
        marker=dict(color=SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row2, col=1)

#exptlight
fig.append_trace(go.Scattergl(x =time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row2, col=2)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean']+df5.loc[df5.ExperimentState == "Full", 'CI'],
        mode='lines',
        marker=dict(color = SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=2)

fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean']-df5.loc[df5.ExperimentState == "Full", 'CI'],
        marker=dict(color = SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row2, col=2)


#wtdark
fig.append_trace(go.Scattergl(x =time2,
        y=df6.loc[df6.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_WT',
        marker = dict(color = "#000"),
        line=dict(width=2),
        showlegend=False    
    ), row=row2, col=1)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time2,
        y=df6.loc[df6.ExperimentState == "Dark", 'mean']+df6.loc[df6.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=1)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time2,
        y=df6.loc[df6.ExperimentState == "Dark", 'mean']-df6.loc[df6.ExperimentState == "Dark", 'CI'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), row=row2, col=1)

#wtlight
fig.append_trace(go.Scattergl(x =time2,
        y=df6.loc[df6.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_WT',
        marker = dict(color = "#000"),
        line=dict(width=2),
        showlegend=False   
    ), row=row2, col=2)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time2,
        y=df6.loc[df6.ExperimentState == "Full", 'mean']+df6.loc[df6.ExperimentState == "Full", 'CI'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=2)

fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time2,
        y=df6.loc[df6.ExperimentState == "Full", 'mean']-df6.loc[df6.ExperimentState == "Full", 'CI'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), row=row2, col=2)


#raster
count_wtfull = counterfall(df_rrfull, nnumber, nnumberwt, "WT")
count_exptdark = counterfall(df_rrdark, nnumber, nnumberwt, "EXPT")
count_wtdark = counterfall(df_rrdark, nnumber, nnumberwt, "WT")
count_exptfull = counterfall(df_rrfull, nnumber, nnumberwt, "EXPT")

fig.append_trace(go.Scatter(
        name='WT',
        x=count_wtdark['value'],
        y=count_wtdark['counts'],
        marker=dict(color="#444"),
        line=dict(width=2),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.1)',
        fill='tozeroy', showlegend=False
    ), row=row3, col=1)

fig.append_trace(go.Scatter(
        name='EXPT',
        x=count_exptdark['value'],
        y=count_exptdark['counts'],
        marker=dict(color=lightcolor),
        line=dict(width=2),
        mode='lines',
        fillcolor=SDinner,
        fill='tozeroy', showlegend=False
    ), row=row3, col=1)

fig.append_trace(go.Scatter(
        name="w1118 controls",
        x=count_wtfull['value'],
        y=count_wtfull['counts'],
        marker=dict(color="#444"),
        line=dict(width=2),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.1)',
        fill='tozeroy', showlegend=True
    ), row=row3, col=2)

fig.append_trace(go.Scatter(
        name=transgenic,
        x=count_exptfull['value'],
        y=count_exptfull['counts'],
        marker=dict(color=lightcolor),
        line=dict(width=2),
        mode='lines',
        fillcolor=SDinner,
        fill='tozeroy', showlegend=True
    ), row=row3, col=2)



#row2
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=90,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row1,col=1)
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=90,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row1,col=2)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro", range=[0,90],row=row1,col=2)
fig.update_yaxes(title='Mean of' +'<br>' + '<b>' + 'Y position (mm)' + '</b>',title_font=dict(size=15), gridcolor = "gainsboro", range=[0,90],tickvals=np.arange(0,91,20),showline=True, linewidth=2, linecolor='black', row=row1,col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro", row=row1,col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro", row=row1,col=2)

#row3
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=100,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row2,col=1)
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=100,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row2,col=2)
fig.update_yaxes(gridcolor = "gainsboro",title='Mean of ' +'<br>' + '<b>' +'Speed (mm/s)'+ '</b>',title_font=dict(size=15),range=[0,40],tickvals=np.arange(0,41,10),showline=True, linewidth=2, linecolor='black', row=row2,col=1)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black',range=[0,40],gridcolor = "gainsboro",tickvals=np.arange(0,41,10),row=row2,col=2)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black',gridcolor = "gainsboro", row=row2,col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro",row=row2,col=2)

#row4
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=300,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row3,col=1)
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=300,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row3,col=2)
fig.update_yaxes(gridcolor = "gainsboro",title='# of '+ '<b>' + 'Falls'+ '</b>',title_font=dict(family="helvetica",size=17), range=[0,15],tickvals=np.arange(0,16,5),showline=True, linewidth=2, linecolor='black',row=row3,col=1)
fig.update_yaxes(showline=True,gridcolor = "gainsboro", linewidth=2, linecolor='black',range=[0,15],tickvals=np.arange(0,16,5),row=row3,col=2)

fig.update_xaxes(title='Seconds(s)',title_font=dict(size = 15), range=[0,23],tickvals=np.arange(0,24,5),showline=True, gridcolor = "gainsboro",linewidth=2, linecolor='black', row=row3,col=1)
fig.update_xaxes(title='Seconds(s)', title_font=dict(size = 15), range=[0,23],tickvals=np.arange(0,24,5),showline=True, gridcolor = "gainsboro",linewidth=2, linecolor='black', row=row3,col=2)


fig.update_layout(title = '<b>' + driver + '</b>' + " (n= " + str(nnumber) + ")", title_font=dict(family="helvetica", size = 20), font = dict(family = "helvetica"), height=600, width=900, hovermode='x unified', showlegend=True, plot_bgcolor='rgba(0,0,0,0)')
fig.update_annotations(font=dict(family="helvetica", size = 17))

#legend
fig.update_layout(showlegend=True, legend=dict(
    title_font_family='helvetica',
    font=dict(
        size=12
    )
))
fig.update_layout(legend= {'itemsizing': 'constant'})

fig.show()

In [21]:
df3 = removingextraline(df_meande)
df4 = removingextraline(df_meandw)
df5 = removingextraline(df_meanse)
df6 = removingextraline(df_meansw)
driver = transgenic

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objects import Layout

fig = make_subplots(rows=6, cols=1, row_heights=[0.02, 0.02, 0.02, 0.3,0.3, 0.3],shared_yaxes=True, shared_xaxes=True,horizontal_spacing = 0.05) #, subplot_titles=("Dark phase", "Light phase", "", " ")

if responder == "ACR":
    barcolr = "#00ff00"
if responder == "Chrimson2":
    barcolr = "#DC143C"

barcolr = "#00ff00"

fig.add_trace(go.Scatter(x=[], y=[]),row=1, col=1)
fig.add_shape(type="rect",x0=0, y0=0, x1=0.2, y1=2,line=dict(color="black",width=1,),fillcolor="gold",row=1,col=1)
fig.add_shape(type="rect",x0=0.2, y0=0, x1=3, y1=1,line=dict(color="black",width=2,),fillcolor="black",row=2,col=1)
fig.add_shape(type="rect",x0=3, y0=0, x1=23, y1=1,line=dict(color="black",width=2,),fillcolor=barcolr,row=3,col=1)
fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,1],row=1, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,2.1],row=1, col=1)

fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,3.05],row=2, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=2, col=1)


fig.update_xaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-2.05,23.05],row=3, col=1)
fig.update_yaxes(showgrid=False,zeroline=False,ticks="",showline=False,showticklabels=False,range=[-0.05,1.05],row=3, col=1)


darkcolor  = "#4B0076"
lightcolor = "#0055ff"
SDcolor = " #cce6ff"
SDinner = "rgba(179, 217, 255,0.3)"

nnumber = int(0.25*(len(dfexpt.columns)-2))
nnumberwt = int(0.25*(len(dfwt.columns)-2))

time = df3.loc[df3.ExperimentState == "Dark", 'Seconds']
time2 = df4.loc[df4.ExperimentState == "Dark", 'Seconds']
#expt_dark
row1 = 4
row2 = 5
row3 = 6
fig.append_trace(go.Scattergl(x =time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='Mean of Y position(mm)_Expt',
        marker = dict(color = darkcolor),
        line=dict(width=2),                                           
        showlegend=False   
    ), row=row1, col=1)

#expt_light
fig.append_trace(go.Scattergl(x =time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean of Y position(mm)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row1, col=1)

#exptdark_CI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean']+df3.loc[df3.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color= darkcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=1)
fig.append_trace(go.Scatter(
        name=transgenic,
        x=time,
        y=df3.loc[df3.ExperimentState == "Dark", 'mean']-df3.loc[df3.ExperimentState == "Dark", 'CI'],
        marker=dict(color= darkcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor= SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row1, col=1)

#exptlight_CI
fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean']+df3.loc[df3.ExperimentState == "Full", 'CI'],
        mode='lines',
        marker=dict(color=SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row1, col=1)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time,
        y=df3.loc[df3.ExperimentState == "Full", 'mean']-df3.loc[df3.ExperimentState == "Full", 'CI'],
        marker=dict(color=SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row1, col=1)

#speed


time = df5.loc[df5.ExperimentState == "Dark", 'Seconds']
time2 = df6.loc[df6.ExperimentState == "Dark", 'Seconds']
#exptdark
fig.append_trace(go.Scattergl(x =time,
        y=df5.loc[df5.ExperimentState == "Dark", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_Expt',
        marker = dict(color = darkcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row2, col=1)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Dark", 'mean']+df5.loc[df5.ExperimentState == "Dark", 'CI'],
        mode='lines',
        marker=dict(color= darkcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=1)
fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=df5['Seconds'],
        y=df5.loc[df5.ExperimentState == "Dark", 'mean']-df5.loc[df5.ExperimentState == "Dark", 'CI'],
        marker=dict(color=darkcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row2, col=1)

#exptlight
fig.append_trace(go.Scattergl(x =time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean'],
        mode='lines',
        name='Mean speed (mm/s)_Expt',
        marker = dict(color = lightcolor),
        line=dict(width=2),
        showlegend=False    
    ), row=row2, col=1)

fig.append_trace(go.Scatter(
        name='Upper Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean']+df5.loc[df5.ExperimentState == "Full", 'CI'],
        mode='lines',
        marker=dict(color = SDcolor),
        line=dict(width=0),
        showlegend=False
    ), row=row2, col=1)

fig.append_trace(go.Scatter(
        name='Lower Bound',
        x=time,
        y=df5.loc[df5.ExperimentState == "Full", 'mean']-df5.loc[df5.ExperimentState == "Full", 'CI'],
        marker=dict(color = SDcolor),
        line=dict(width=0),
        mode='lines',
        fillcolor=SDinner,
        fill='tonexty',
        showlegend=False
    ), row=row2, col=1)


#raster
count_exptdark = counterfall(df_rrdark, nnumber, nnumberwt, "EXPT")
count_exptfull = counterfall(df_rrfull, nnumber, nnumberwt, "EXPT")

fig.append_trace(go.Scatter(
        name='EXPT',
        x=count_exptdark['value'],
        y=count_exptdark['counts'],
        marker=dict(color=darkcolor),
        line=dict(width=2),
        mode='lines',
        fillcolor=SDinner,
        fill='tozeroy', showlegend=False
    ), row=row3, col=1)

fig.append_trace(go.Scatter(
        name=transgenic,
        x=count_exptfull['value'],
        y=count_exptfull['counts'],
        marker=dict(color=lightcolor),
        line=dict(width=2),
        mode='lines',
        fillcolor=SDinner,
        fill='tozeroy', showlegend=True
    ), row=row3, col=1)


#row2
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=90,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row1,col=1)

fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro", range=[0,90],row=row1,col=1)
fig.update_yaxes(title='Mean of' +'<br>' + '<b>' + 'Y position (mm)' + '</b>',title_font=dict(size=15), gridcolor = "gainsboro", range=[0,90],tickvals=np.arange(0,91,20),showline=True, linewidth=2, linecolor='black', row=row1,col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor = "gainsboro", row=row1,col=1)


#row3
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=100,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row2,col=1)

fig.update_yaxes(gridcolor = "gainsboro",title='Mean of ' +'<br>' + '<b>' +'Speed (mm/s)'+ '</b>',title_font=dict(size=15),range=[0,40],tickvals=np.arange(0,41,10),showline=True, linewidth=2, linecolor='black', row=row2,col=1)
# fig.update_yaxes(showline=True, linewidth=2, linecolor='black',range=[0,40],gridcolor = "gainsboro",tickvals=np.arange(0,41,10),row=row2,col=2)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black',gridcolor = "gainsboro", row=row2,col=1)

#row4
fig.add_shape(type="line",x0=3, y0=0, x1=3, y1=300,line=dict(color="black",width=3,dash = "dot"),fillcolor="black", row=row3,col=1)
fig.update_yaxes(gridcolor = "gainsboro",title='# of '+ '<b>' + 'Falls'+ '</b>',title_font=dict(family="helvetica",size=17), range=[0,15],tickvals=np.arange(0,16,5),showline=True, linewidth=2, linecolor='black',row=row3,col=1)
#fig.update_yaxes(showline=True,gridcolor = "gainsboro", linewidth=2, linecolor='black',range=[0,15],tickvals=np.arange(0,16,5),row=row3,col=2)
fig.update_xaxes(title='Seconds(s)',title_font=dict(size = 15), range=[0,23],tickvals=np.arange(0,24,5),showline=True, gridcolor = "gainsboro",linewidth=2, linecolor='black', row=row3,col=1)
fig.update_layout(title = '<b>' + driver + '</b>' + " (n= " + str(nnumber) + ")", title_font=dict(family="helvetica", size = 20), font = dict(family = "helvetica"), height=600, width=600, hovermode='x unified', showlegend=True, plot_bgcolor='rgba(0,0,0,0)')
fig.update_annotations(font=dict(family="helvetica", size = 17))

#legend
fig.update_layout(showlegend=False, legend=dict(
    title_font_family='helvetica',
    font=dict(
        size=12
    )
))
fig.update_layout(legend= {'itemsizing': 'constant'})

fig.show()

In [49]:
savelocation = openPath + "images\\"
fig.write_image(savelocation + "lineplot of " + transgenic +".jpg", scale=2)